# 🚢 Titanic Estimator

6-step ML framework <a href="https://www.mrdbourke.com/a-6-step-field-guide-for-building-machine-learning-projects/">here</a>:

1️⃣ **Problem Definition:** What problem are we trying to solve? 

2️⃣ **Data:** What data do we have?

3️⃣ **Evaluation:** What defines success? What metric will we use to evaluate our model? 

4️⃣ **Features:** What features should we use?

5️⃣ **Model:** What model will weuse to solve the problem?

6️⃣ **Experimentation:** What have we tried and what else can we try?


### 1. Problem Definition

Predicting the surviving passengers of the Titanic shipwreck based on the features of the passengers.

### 2. Data

Data comes from <a href="https://www.kaggle.com/competitions/titanic/overview">Kaggle</a>. See the data dictionary for detailed descriptions of the features.

### 3. Evaluation

Accuracy of the model, evaluated when submitting to the Kaggle competition.

### 4. Features

In [4]:
# Data exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Machine learning
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

### 5. Model

### 6. Experimentation